In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib.animation import FuncAnimation

plt.style.use('default')

In [2]:
@jit(nopython=True)
def random_walk_2d_n8(nwalks):
    # Arrays to store x, y coordinates
    x = np.zeros(nwalks)
    y = np.zeros(nwalks)
    
    # 8 directions
    n8 = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
    for n in range(nwalks):
        idx = np.random.choice(len(n8), 1)[0]
        x[n] = x[n - 1] + n8[idx][0]
        y[n] = y[n - 1] + n8[idx][1]          
    return x, y

In [3]:
def closest_node(node, offsets):
    distances = np.sum((offsets - node) ** 2, axis=1)
    return np.argmin(distances)

In [4]:
def animate(i):
    # Update point position
    pt_plt.set_data(x[i], y[i])
    
    # Update Hexbin counts
    closest_idx = closest_node((x[i], y[i]), offsets)
    counts[closest_idx] += 1
    hb_plt.set_array(counts)
    
    # Update Hexbin label
    label_i = labels[closest_idx]
    label_i.set_text(int(counts[closest_idx]))
    return pt_plt, hb_plt, 

In [5]:
nwalks = 250
x, y = random_walk_2d_n8(nwalks)

In [6]:
# Plot the result
fig, ax = plt.subplots()
pt_plt, = ax.plot([], [], marker="o", 
                  mfc = 'white', 
                  mec = 'tab:blue', 
                  zorder=2)
# Start point
ax.scatter(x[0], y[0], color="white", edgecolors="k", zorder=2)

# End point
ax.scatter(x[-1], y[-1], color="tab:red", zorder=2)

# Path
ax.plot(x, y, color="k", lw=0.1, zorder=1)

# Initialize hexbin
hb_plt = ax.hexbin(x, y, gridsize=10, cmap="OrRd")
offsets = hb_plt.get_offsets()
offsets_x = offsets[:, 0]
offsets_y = offsets[:, 1]
counts = hb_plt.get_array() * 0
hb_plt.set_array(counts)

# Initialize hexbin labels
labels = []
for off in offsets:
    labels.append(ax.text(off[0], off[1], "", 
                           ha="center", 
                           va="center", 
                           c="tab:orange",
                           weight="bold",
                           path_effects=[pe.withStroke(linewidth=2, 
                                                       foreground="white")]))
plt.axis('off')
ax.set(xlim = (x.min() - 1.5, x.max() + 1.5), 
       ylim = (y.min() - 1.5, y.max() + 1.5))
plt.tight_layout()
plt.close()

In [ ]:
# Create the animation
animation = FuncAnimation(fig, animate, frames=len(x))
animation.save('04_Hexagons.gif', dpi=300)